In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

from skimage import filters 
from skimage.morphology import disk
from skimage.color import hsv2rgb
from tifffile import imsave
from PIL import Image


In [2]:
def iba1_mask (img, saveto , filename):
    enhanced = filters.unsharp_mask (img , radius = 20.0 , amount = 2 ) #sharpens the image and reduces background 
    otsu_thr = filters.threshold_otsu (enhanced) #set threshold
    otsu = enhanced > otsu_thr #otsu as array 
    mask = filters.median (otsu , disk (2)) #reduce noise 
    
    im = Image.fromarray (mask)
    im.save (saveto + "/" + filename.replace ("tif" ,"png" ))
    print ("Titel:\t" , saveto + "/" + filename.replace ("tif" ,"png" ))

### Iba1 images in "rootdir" folder are thresholded (function "iba1_mask") and masks are saved

In [ ]:
rootdir = r"G:\human DRG\tif_data\splitted"  
for dirpath , dirnames , filenames in os.walk (rootdir) : 
    if dirpath.endswith('Iba1'):
        for filename in filenames:
            img_path = os.path.join (dirpath, filename)
            saveto = dirpath + "_pred/masks"
            
            try: #makes directory if it doesnt exist
                os.makedirs (saveto)
            except:
                pass
            img = plt.imread(img_path) 
            iba1_mask(img, saveto , filename)
            

### substract NF mask from Iba1 image before thresholding

In [3]:
def get_iba1_mask (img, savepath):
    enhanced = filters.unsharp_mask (img , radius = 20.0 , amount = 2 ) #sharpens the image and reduces background 
    otsu_thr = filters.threshold_otsu (enhanced) #set threshold
    otsu = enhanced > otsu_thr #otsu as array 
    mask = filters.median (otsu , disk (2)) #reduce noise 
    
    im = Image.fromarray (mask)
    im.save (savepath)
    print ("Titel:\t" , savepath)

In [ ]:
rootdir = r"G:\human DRG\tif_data\splitted"  
for dirpath , dirnames , filenames in os.walk (rootdir) : 
    if dirpath.endswith('Iba1'):
        
        saveto_dir = dirpath + "_pred/masks_adj"           
        nf_mask_dir = saveto_dir.replace ("Iba1_pred" , "NF_pred")[:-4]
        print(saveto_dir)
        print(nf_mask_dir)
  
        try: #makes directory if it doesnt exist
            os.makedirs (saveto_dir)
        except:
            pass
        
        for filename in filenames:
            iba1_path = os.path.join (dirpath, filename)
            filename_png = filename.replace ("tif" ,"png" )
            nf_mask_path = os.path.join (nf_mask_dir, filename_png)
            iba1_mask_path = os.path.join (saveto_dir, filename_png)
            
            iba1 = plt.imread(iba1_path) 
            nf_mask = plt.imread(nf_mask_path)
            
            # set iba1 image where neurons are to zero
            iba1_adj = iba1.copy()
            iba1_adj[nf_mask>0] = 0
            
            get_iba1_mask(iba1_adj, iba1_mask_path)